# Tutorial for the simulation

The functions relevants are packaged into Kagome.py
The simulation class can be initialised just from the lattice number of triangles in the two directions Nx and Ny
and specifying J for the hamiltonian , We also provide a name for the directory in which the files will be created for the results
- the next two argument enables you to control the seed for the generation of the initial configuration 
-the last two arguments enables you to provide a starting spin configuration to continue an old simulation

In [3]:
import Kagome


J=1
L=5
Nx=L
Ny=L
diretcory='tutorial'
A=Kagome.Configuration(Nx,Ny, J,diretcory,seed_control=False,seed=1500,preload=False,oldconfig=None)


Then we can start the simulation by looping over the temperatures we first specify the number of hybrid monte carlo steps for decoherence time where we do not save the 
energy for heat capacity measurements after changing the temperature 

The monte carlo function is where we can run the simulation first argument is the number of MC steps  second is the temperature for the metropolis hastings acceptation rate then we specify if we want to record the energy every 1000 steps for the heat capacity measurement with the measure_capa boolean argument
Next if we want those steps to be hybrid with overrelaxation we specify overrelaxation=True and finally the overrelaxation_dose is the fraction of the lattice that will be 
rotated by an angle pi around the mean field
- we can then call the function saveconfig with the name of the file to be created with the informations

In [4]:
Temp=[0.5,0.4,0.3]

decoherencetime=1000
numberMC=2000


for j,T in enumerate(Temp):
 
    A.Monte_Carlo(decoherencetime,T,measure_capa=False,overrelaxation=True,overrelaxation_dose=0.1)

    A.Monte_Carlo(numberMC,T,measure_capa=True,overrelaxation=True,overrelaxation_dose=0.1)


    name=f'simulationtempsetp{j}'
    
    A.saveconfig(name,T)

In [ ]:
    
    
    
J = 1
L = 12
Nx = L
Ny = L
theta = 2*np.pi/3

T_high = 1
T_low = 0.002
n_T = 20
Temp = np.linspace(T_high, T_low, n_T)



# Runs to get the mean values and standard deviations of the physical quantities
n_runs = 15  

E_mean_all = []
E_std_all = []
Capa_mean_all = []
Capa_std_all = []



for j, T in enumerate(Temp):
    decoherencetime = 50000
    numberMC = 100000

    Ener_runs = []
    Capa_runs = []

    for run in range(n_runs):
        A = Configuration(1, theta, Nx, Ny, J)

        A.Monte_Carlo(decoherencetime, T)
        A.verify_norm()

        Ener = A.measure_Capa(numberMC, T, Nx, Ny)
        A.verify_norm()

        Ener_runs.append(np.mean(Ener))
        Capa_runs.append(A.Capa[-1])

    E_mean_all.append(np.mean(Ener_runs))
    E_std_all.append(np.std(Ener_runs))
    
    Capa_mean = np.mean(Capa_runs)
    Capa_std = np.std(Capa_runs)
    
    # filtering because of a point of anomaly
    Capa_mean = min(Capa_mean, 2)
    Capa_std = min(Capa_std, 2 - Capa_mean)


    Capa_mean_all.append(Capa_mean)
    Capa_std_all.append(Capa_std)

    print(f"T step {j+1}/{n_T} done")
    
    
    
    
A.display_config()



triangle_sums = A.triangle_spins_sum()

for i in range(A.Nx):
    for j in range(A.Ny):
        print(f"Sum of spins in the triangle ({i},{j}) :", triangle_sums[i, j])   


# Plot Heat Capacity Vs T in browser with plotly

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Temp,
    y=Capa_mean_all,
    error_y=dict(
        type='data',
        array=Capa_std_all,
        visible=True
    ),
    mode='markers+lines',
    name="Capacité thermique"
))

fig.update_layout(
    xaxis_type="log",
    xaxis_title="Température T",
    yaxis_title="Capacité thermique C(T)",
    title="Capacité thermique interactive",
    template="plotly_white"
)

fig.show(renderer="browser")




# Plot Energy Vs T in browser with plotly


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=Temp,
    y=E_mean_all,
    error_y=dict(
        type='data',
        array=E_std_all,
        visible=True
    ),
    mode='markers+lines',
    name="Heat Capacity"
))

fig.update_layout(
    xaxis_title="Temperature T",
    yaxis_title="Energy E",
    title="Interactive Energy",
    template="plotly_white"
)

fig.show(renderer="browser")
